## Machine Learning Record Mining

Project to create a pipeline that uses GeoDeepDive's output to find Unaquired Sites for Neotoma.

Using NLP parsed text and a Data Science approach, identify whether a paper is suitable for Neotoma and detect features such as 'Site Name', 'Location', 'Age Span' and 'Site Descriptions'.

In [3]:
# Loading libraries

import numpy as np
import pandas as pd
import csv
import psycopg2
import re

# Load Postgres Server
from src.config import config

In [4]:
# Options for DF display
#pd.set_option('display.max_colwidth', 50)
#pd.set_option('display.max_rows', 10)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

## Loading and viewing the Data

### Loading NLP Sentences

In [5]:
# Connect to PostgreSQL server from terminal:
# pg_ctl -D PSQL_Data -l logfile start

try:
    params = config()
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(**params)
    # Create a new cursor
    cur = conn.cursor()
    nlp_sentences = pd.read_sql_query('''SELECT * FROM sentences;''', conn)
    # Close the cursor and connection to so the server can allocate
    # bandwidth to other requests
    cur.close()
    conn.close()
    print('Data obtained from PostgreSQL')
    
# If no SQL db, load from a file
except:
    header_list = ["_gddid", "sentid", "wordidx", "words", "part_of_speech", "special_class", 
               "lemmas", "word_type", "word_modified"]
    nlp_sentences = pd.read_csv("../Do_not_commit_data/sentences_nlp352", sep='\t', names = header_list)
    nlp_sentences = nlp_sentences.replace('"', '', regex = True)\
                                 .replace('\{', '', regex = True)\
                                 .replace('}', '', regex = True)\
                                 .replace(',', ',', regex = True)
    nlp_sentences['wordidx']= nlp_sentences['wordidx'].str.split(",")
    nlp_sentences['words']= nlp_sentences['words'].str.split(",")
    nlp_sentences['poses']= nlp_sentences['poses'].str.split(",")
    nlp_sentences['ners']= nlp_sentences['ners'].str.split(",")
    nlp_sentences['lemmas']= nlp_sentences['lemmas'].str.split(",")
    nlp_sentences['dep_paths']= nlp_sentences['dep_paths'].str.split(",")
    nlp_sentences['dep_parents']= nlp_sentences['dep_parents'].str.split(",")
    print('Data obtained from text file')

Data obtained from PostgreSQL


In [6]:
nlp_sentences.head(3)

,_gddid,sentid,wordidx,words,part_of_speech,special_class,lemmas,word_type,word_modified
0,54b43266e138239d8684efed,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, ...]","[Available, online, at, www.sciencedirect.com, Quaternary, Research, 69, -LRB-, 2008, -RRB-, 263, --, 275, www.elsevier.com/locate/yqres, Development, of, the, mixed, conifer, forest, in, northern, New, Mexico, and, its, relationship, to, Holocene, environmental, change, R., Scott, Anderson, a, ,, b, ,, ,, Renata, B., Jass, b, ,1, ,, Jaime, L., Toney, b, ,2, ,, Craig, D., Allen, c, ,, Luz, M., Cisneros-Dozal, d, ,, Marcey, Hess, d, ,, Jeff, Heikoop, d, ,, Julianna, Fessenden, d, a, Center, for, Environmental, Sciences, &, Education, ,, Box, 5694, ,, Northern, Arizona, University, ,, Flagstaff, ,, AZ, 86011, ,, USA, b, Quaternary, Sciences, Program, &, Bilby, Research, ...]","[JJ, NN, IN, NNP, NNP, NNP, CD, -LRB-, CD, -RRB-, CD, :, CD, NNS, NN, IN, DT, JJ, NN, NN, IN, JJ, NNP, NNP, CC, PRP$, NN, TO, NNP, JJ, NN, NNP, NNP, NNP, DT, ,, NN, ,, ,, NNP, NNP, NNP, NN, CD, ,, NNP, NNP, NNP, NN, CD, ,, NNP, NNP, NNP, NN, ,, NNP, NNP, NNP, NN, ,, NNP, NNP, NN, ,, NNP, NNP, NN, ,, NNP, NNP, NN, DT, NNP, IN, NNP, NNPS, CC, NNP, ,, NNP, CD, ,, NNP, NNP, NNP, ,, NNP, ,, NNP, CD, ,, NNP, NN, NNP, NNP, NNP, CC, NNP, NNP, ...]","[O, O, O, O, O, O, NUMBER, O, DATE, O, NUMBER, O, NUMBER, O, O, O, O, O, O, O, O, O, LOCATION, LOCATION, O, O, O, O, O, O, O, PERSON, PERSON, PERSON, O, O, O, O, O, PERSON, PERSON, PERSON, O, NUMBER, O, PERSON, PERSON, PERSON, O, NUMBER, O, PERSON, PERSON, PERSON, O, O, PERSON, PERSON, PERSON, O, O, PERSON, PERSON, O, O, PERSON, PERSON, O, O, PERSON, PERSON, O, O, ORGANIZATION, ORGANIZATION, ORGANIZATION, ORGANIZATION, ORGANIZATION, ORGANIZATION, O, O, NUMBER, O, ORGANIZATION, ORGANIZATION, ORGANIZATION, O, LOCATION, O, LOCATION, NUMBER, O, ORGANIZATION, ORGANIZATION, ORGANIZATION, ORGANIZATION, ORGANIZATION, ORGANIZATION, ORGANIZATION, ORGANIZATION, ...]","[available, online, at, www.sciencedirect.com, Quaternary, Research, 69, -lrb-, 2008, -rrb-, 263, --, 275, www.elsevier.com/locate/yqres, development, of, the, mixed, conifer, forest, in, northern, New, Mexico, and, its, relationship, to, Holocene, environmental, change, R., Scott, Anderson, a, ,, b, ,, ,, Renata, B., Jass, b, ,1, ,, Jaime, L., Toney, b, ,2, ,, Craig, D., Allen, c, ,, Luz, M., Cisneros-Dozal, d, ,, Marcey, Hess, d, ,, Jeff, Heikoop, d, ,, Julianna, Fessenden, d, a, Center, for, Environmental, Sciences, &, Education, ,, Box, 5694, ,, Northern, Arizona, University, ,, Flagstaff, ,, AZ, 86011, ,, USA, b, Quaternary, Sciences, Program, &, Bilby, Research, ...]","[dep, dep, dep, dep, dep, dep, dep, , dep, , dep, , dep, dep, dep, dep, dep, dep, dep, dep, dep, dep, dep, dep, cc, dep, dep, dep, dep, dep, dep, dep, dep, dep, dep, , dep, , , dep, dep, dep, dep, dep, , dep, dep, dep, dep, dep, , dep, dep, dep, dep, , dep, dep, dep, dep, , dep, dep, dep, , dep, dep, dep, , dep, dep, dep, dep, dep, dep, dep, dep, cc, dep, , dep, dep, , dep, dep, dep, , dep, , dep, dep, , dep, dep, dep, dep, dep, cc, dep, dep, ...]","[218, 218, 218, 218, 218, 218, 218, 0, 218, 0, 218, 0, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 0, 218, 0, 0, 218, 218, 218, 218, 218, 0, 218, 218, 218, 218, 218, 0, 218, 218, 218, 218, 0, 218, 218, 218, 218, 0, 218, 218, 218, 0, 218, 218, 218, 0, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 0, 218, 218, 0, 218, 218, 218, 0, 218, 0, 218, 218, 0, 218, 218, 218, 218, 218, 218, 218, 218, ...]"
1,54b43266e138239d8684efed,2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[T

In [7]:
# Running this takes forever. Do not run unless necessary
# nlp_sentences.describe(include = 'all')

In [8]:
avg_sentences = nlp_sentences.groupby('_gddid')['_gddid','sentid'].transform('max')
avg_sentences.drop_duplicates(subset='_gddid', keep='first', inplace=True)
avg_sentences.describe()

,sentid
count,218.000000
mean,513.600917
std,232.460891
min,2.000000
25%,397.250000
50%,504.000000
75%,638.250000
max,1330.000000


From the Summary Statistics, despite having 111,965 sentences, there are only 218 unique articles (`_gddid`). The article with most sentences, had 1330 sentences; that's the max number found in `sentid`.  
In order to know the average of sentences, I grouped by `_gddid` and then, obtained the number of sentences per article. After that, I just got the mean through a Summary of statistics to know that on average, each article has 514 sentences.

### Loading Bibliography Data

In [9]:
import json
with open('../Do_not_commit_data/bibjson', 'r') as f:
    bib_dict = json.load(f)

In [10]:
# Normalizing data so that we have access to the 'identifier'

# TODO Load into SQL server and connect through SQL
bibliography = pd.io.json.json_normalize(bib_dict,'identifier',['publisher', 'title', ['journal','name', 'name'], ['author'], 'year', 'number', 'volumne', ['link'], '_gddid', 'type', 'pages'], record_prefix='_', errors='ignore')

bibliography['link'] = bibliography['link'].astype(str)

url = bibliography['link'].str.split(", ", expand = True)

bibliography['link_url'] = url[0]
bibliography['link_type'] = url[1]

bibliography['link_url'] = bibliography['link_url'].replace("\[{'url': '", "", regex = True)\
                                                   .replace("'", "", regex = True)

bibliography['link_type'] = bibliography['link_type'].replace("'type': '", "", regex = True)\
                                                     .replace("'}]", "", regex = True)

bibliography['author'] = bibliography['author'].astype(str)
bibliography['author'] = bibliography['author'].replace("\[{'name': '", "", regex = True)\
                                               .replace("{'name': '", "", regex = True)\
                                               .replace("'},", ";", regex = True)\
                                               .replace("'}]", "", regex = True)
bibliography = bibliography[['_type', '_id', 'publisher', 'title', 'journal.name.name',	'author', 'year', 'number', 'volumne', '_gddid', 'type', 'pages', 'link_url', 'link_type']]

bibliography.head(3)

,_type,_id,publisher,title,journal.name.name,author,year,number,volumne,_gddid,type,pages,link_url,link_type
0,doi,10.1016/S0277-3791(99)00007-4,Elsevier,"Palaeoclimate, chronology and vegetation history of the Weichselian Lateglacial: comparative analysis of data from three cores at Lago Grande di Monticchio, southern Italy",Quaternary Science Reviews,"Huntley, B.; Watts, W.A.; Allen, J.R.M.; Zolitschka, B.",1999,7,NaN,550453fde1382326932d85f7,article,945--960,http://www.sciencedirect.com/science/article/pii/S0277379199000074,publisher
1,doi,10.1139/b92-002,Canadian Science Publishing,"Holocene history of forest trees on the Bruce Peninsula, southern Ontario",Canadian Journal of Botany,"Bennett, K. D.",1992,1,NaN,578b5aabcf58f1587003efba,article,6--18,http://www.nrcresearchpress.com/doi/abs/10.1139/b92-002,publisher
2,doi,10.1016/0031-0182(80)90043-7,Elsevier,Glacial sequence and environmental history in the Sierra Nevada del cocuy (Colombia),"Palaeogeography, Palaeoclimatology, Palaeoecology","Van Der Hammen, T.; Barelds, J.; De Jong, H.; De Veer, A.A.",1980,NaN,NaN,54b43244e138239d868493cd,article,247--340,http://www.sciencedirect.com/science/article/pii/0031018280900437,publisher


In [11]:
bibliography.describe(include = 'all')

,_type,_id,publisher,title,journal.name.name,author,year,number,volumne,_gddid,type,pages,link_url,link_type
count,221,221,221,221,221,221,221,183,0,221,221,216,221,220
unique,1,221,6,221,41,213,47,28,0,221,1,214,221,1
top,doi,10.1016/j.foreco.2007.10.008,Elsevier,Holocene climate variability in northernmost Europe,Quaternary Research,"Markgraf, Vera",2011,3,NaN,5714568acf58f125068458dc,article,1--13,http://doi.wiley.com/10.1111/j.1502-3885.2010.00159.x,publisher
freq,221,1,138,1,44,3,14,40,NaN,1,221,2,1,220


## EDA

Reviewing our data includes skimming through some papers online and seeing if the data is consistent with our NLP Sentences dataframe.

From there, we can also visualize what we would like our model to predict: 'Location', 'Site Name', 'Age Span', and 'Site Description' from a "Human perspective".

### Finding Latitude and Longitud via REGEX 

After reviewing some PDF's and viewing the data parsing, I noticed some possible patterns for coordinates.

<img src="figures/img/00_snippet_of_pdf.png" alt="Drawing" style="width: 300px;"/>

Which translates to our pandas dataframe as:

![alt text](figures/img/01_snippet_in_sql.png "PandasVersion")

As a first problem, we can notice that the degrees symbol `°` was translated as a `3`. On some other occassions, it is confused as an `o` or a middle point `◦`. For `'`, this symbol got confused by the word `cents`.

I am still going to use normal REGEX to find coordinates in the most intuitive way. 

A coordinate for longitude is a number ranging from -180° to 180°. It has three components, `°, ',"` and it covers the position in E or W.

Latitude goes from -90° to 90°, has the same three components and goes from S to N.

Using REGEX, and assuming data consistency, the following code should be able to extract coordinates. 

In [12]:
dms_regex = r"([-]?\d{1,3}[°|′|\'|,]{0,3}\d{1,2}[,|′|\']{1,3}\d{0,2}[,|′|\']{1,4}[NESWnesw]?[\s|,|\']+?[-]?\d{1,3}[°|,|′|\']+\d{1,2}[,|′|\']+\d{0,2}[,|′|\'][,|′|\']{0,4}[NESWnesw]?)"

dd_regex = "[\\{,][-]?[1]?[0-9]{1,2}\\.[0-9]{1,}[,]?[NESWnesw],"

In [13]:
# TODO Find REGEX of couples of numbers (123, 234)
nlp_sentences['words_as_string'] = nlp_sentences['words'].apply(lambda x: ','.join(map(str, x)))
nlp_sentences['dms_regex'] = nlp_sentences['words_as_string'].str.findall(dms_regex)


#nlp_sentences[['dms_regex', 'words']]


#nlp_sentences['dms_regex'] = nlp_sentences['dms_regex'].astype(str)
#nlp_sentences['dms_regex'] = nlp_sentences['dms_regex'].replace(",", ";", regex = True)#\
                                                       #.replace(",", "", regex = True)

nlp_sentences[['words_as_string', 'words', 'dms_regex']].head(3)

,words_as_string,words,dms_regex
0,"Available,online,at,www.sciencedirect.com,Quaternary,Research,69,-LRB-,2008,-RRB-,263,--,275,www.elsevier.com/locate/yqres,Development,of,the,mixed,conifer,forest,in,northern,New,Mexico,and,its,relationship,to,Holocene,environmental,change,R.,Scott,Anderson,a,,,b,,,,,Renata,B.,Jass,b,,1,,,Jaime,L.,Toney,b,,2,,,Craig,D.,Allen,c,,,Luz,M.,Cisneros-Dozal,d,,,Marcey,Hess,d,,,Jeff,Heikoop,d,,,Julianna,Fessenden,d,a,Center,for,Environmental,Sciences,&,Education,,,Box,5694,,,Northern,Arizona,University,,,Flagstaff,,,AZ,86011,,,USA,b,Quaternary,Sciences,Program,&,Bilby,Research,Center,,,Box,6013,,,Northern,Arizona,University,,,Flagstaff,,,AZ,86011,,,USA,c,U.S.,Geological,Survey,,,Jemez,Mountains,Field,Station,,,Bandelier,National,Monument,,,HCR-1,,,Box,1,,,#,15,,,Los,Alamos,,,NM,87544,,,USA,d,Earth,&,Environmental,Sciences,Division,,,Hydrology,,,Geochemistry,&,Geology,Group,,,EES-6,,,MS-D462,,,Los,Alamos,National,Laboratory,,,Los,Alamos,,,NM,87545,,,USA,Received,28,April,2007,Available,online,29,January,2008,Abstract,Chihuahueños,Bog,-LRB-,2925,m,-RRB-,in,the,Jemez,Mountains,of,northern,New,Mexico,contains,one,of,the,few,records,of,late-glacial,and,postglacial,development,of,the,mixed,conifer,forest,in,southwestern,North,America,.","[Available, online, at, www.sciencedirect.com, Quaternary, Research, 69, -LRB-, 2008, -RRB-, 263, --, 275, www.elsevier.com/locate/yqres, Development, of, the, mixed, conifer, forest, in, northern, New, Mexico, and, its, relationship, to, Holocene, environmental, change, R., Scott, Anderson, a, ,, b, ,, ,, Renata, B., Jass, b, ,1, ,, Jaime, L., Toney, b, ,2, ,, Craig, D., Allen, c, ,, Luz, M., Cisneros-Dozal, d, ,, Marcey, Hess, d, ,, Jeff, Heikoop, d, ,, Julianna, Fessenden, d, a, Center, for, Environmental, Sciences, &, Education, ,, Box, 5694, ,, Northern, Arizona, University, ,, Flagstaff, ,, AZ, 86011, ,, USA, b, Quaternary, Sciences, Program, &, Bilby, Research, ...]",[]
1,"The,Chihuahueños,Bog,record,extends,to,over,15,000,cal,yr,BP,.","[The, Chihuahueños, Bog, record, extends, to, over, 15,000, cal, yr, BP, .]",[]
2,"An,Artemisia,steppe,,,then,an,open,Picea,woodland,grew,around,a,small,pond,until,ca.,11,700,cal,yr,BP,when,Pinus,ponderosa,became,established,.","[An, Artemisia, steppe, ,, then, an, open, Picea, woodland, grew, around, a, small, pond, until, ca., 11,700, cal, yr, BP, when, Pinus, ponderosa, became, established, .]",[]


In [14]:
#nlp_sentences[['words_as_string', 'words', 'dms_regex']].describe()

The `dms_regex` only returned 95 unique values. That's to say that from the 111965 sentences, only 95 possess something similar to the regex we are looking for.  

Let's see an example of a regex straction as a sanity check that we are getting the values that we need. 

In [15]:
italy_example = nlp_sentences[nlp_sentences['_gddid'] == '550453fde1382326932d85f7']
italy_example = italy_example[italy_example['sentid'] == 10]
italy_example[['words', 'words_as_string', 'dms_regex']]

,words,words_as_string,dms_regex
94875,"[Introduction, Laghi, di, Monticchio, are, two, maar, lakes, located, in, the, explosion, crater, that, lies, west, of, the, peak, of, Monte, Vulture, ,, near, Mel, '', in, the, Basilicata, region, of, southern, Italy, -LRB-, 403, 56, 40, N, ,, 153, 36, 48, E, -RRB-, -LRB-, Fig., 1, -RRB-, .]","Introduction,Laghi,di,Monticchio,are,two,maar,lakes,located,in,the,explosion,crater,that,lies,west,of,the,peak,of,Monte,Vulture,,,near,Mel,'',in,the,Basilicata,region,of,southern,Italy,-LRB-,403,56,40,N,,,153,36,48,E,-RRB-,-LRB-,Fig.,1,-RRB-,.","[403,56,40,N,,,153,36,48,E]"


### Problems with the REGEX approach

As seen before, extracting information using REGEX is very complicated. We would require a lot of queries that would retrieve each particular case.

A possible alternative could be using NER, Named Entity Recognition.

In [16]:
def order_article(article_id):
    '''
    Function to find an article by its gddid in the NLP sentences and have it displayed in order
    
    Keyword arguments:
    article_id -- gddid
    
    Returns:
    article ordered by sentence index
    '''
    article = nlp_sentences[nlp_sentences['_gddid'] == article_id]
    return article[['sentid', 'words']].sort_values(by = 'sentid')

Article 01  
Skimmed info: 
 * `Article Name:` Palaeoclimate_chronology_and_vegetation_history_of_the_Weichselian_Lateglacial
 * `Link`: http://www.sciencedirect.com/science/article/pii/S0277379199000074  
 * `Site Name`:  Laghi di Monticchio
 * `Location`:  40° 56' 40" N, 15° 36' 48" E
 * `Age Span`: 
 * `Site Descriptions`: The record from Lago Grande di Monticchio now adds macrofossil evidence of the presence of relatively boreal tree taxa during the Lateglacial interstadial and provides a chronology based upon annually laminated sediments.

In [17]:
# Article 01 
order_article('550453fde1382326932d85f7')

,sentid,words
94865,1,"[Quaternary, Science, Reviews, 18, -LRB-, 1999, -RRB-, 945, -RCB-, 960, Palaeoclimate, ,, chronology, and, vegetation, history, of, the, Weichselian, Lateglacial, :, comparative, analysis, of, data, from, three, cores, at, Lago, Grande, di, Monticchio, ,, southern, Italy, B., Huntley, *, ,, W.A., Watts, ,, J.R.M., Allen, ,, B., Zolitschka, Department, of, Biological, Sciences, ,, Environmental, Research, Centre, ,, University, of, Durham, ,, South, Road, ,, Durham, DH1, 3LE, ,, UK, School, of, Botany, ,, Trinity, College, ,, Dublin, 2, ,, Ireland, PB, 3.3, -RCB-, Sedimente, und, Beckenbildung, ,, GeoForschungsZentrum, Potsdam, ,, Telegrafenberg, ,, D-14473, Potsdam, ,, Germany, Abstract, Palynological, and, chronological, ...]"
94866,2,"[Plant, macrofossil, data, from, one, core, provide, complementary, evidence, of, the, palaeovegetation, .]"
94867,3,"[Quantitative, palaeoclimate, reconstructions, are, made, from, the, pollen, data, using, pollen-climate, response, surfaces, .]"
94868,4,"[The, chronological, framework, for, Lateglacial, events, as, well, as, the, palaeoclimate, are, compared, with, other, sources, of, evidence, of, Lateglacial, palaeoenvironments, .]"
94870,5,"[Surface, conditions, in, the, North, Atlantic, appear, to, be, more, directly, linked, to, climatic, conditions, in, Italy, than, is, the, temperature, in, Greenland, .]"
94871,6,"[Nonetheless, ,, the, palynological, record, exhibits, high-frequency, variability, that, re, #ects, high-frequency, climatic, changes, paralleling, those, seen, in, the, ice, core, records, from, Greenland, .]"
94872,7,"[1999, Elsevier, Science, Ltd., .]"
94873,8,"[All, rights, reserved, .]"
94874,9,"[1, .]"
94875,10,"[Introduction, Laghi, di, Monticchio, are, two, maar, lakes, located, in, the, explosion, crater, that, lies, west, of, the, peak, of, Monte, Vulture, ,, near, Mel, '', in, the, Basilicata, region, of, southern, Italy, -LRB-, 403, 56, 40, N, ,, 153, 36, 48, E, -RRB-, -LRB-, Fig., 1, -RRB-, .]"


Article 02  
Skimmed info:  
* `Article:` Development of the Mixed Connifer Forest in Northern New Mexico and its relationship to Holocene environmental change
* `Link`: http://www.sciencedirect.com/science/article/pii/S0033589407001512
* `Site Name`:  Chihuahuenos Bog 
* `Location`:  ??? No Given Coordinates 
* `Age Span`:   
* `Site Descriptions`:  Chihuahueños Bog is located within the mixed conifer forest and has affinities to the flora of the southern Rocky Mountains(Foxx and Tierney, 1985; Allen, 2004). Although the area immediately surrounding the bog has been heavily logged in recent years, nearly every conifer species growing in montaneforests of the Jemez mountains is represented either along theshore of the bog or in the surrounding uplands.  

In [18]:
# Article 2
order_article('54b43266e138239d8684efed')

,sentid,words
0,1,"[Available, online, at, www.sciencedirect.com, Quaternary, Research, 69, -LRB-, 2008, -RRB-, 263, --, 275, www.elsevier.com/locate/yqres, Development, of, the, mixed, conifer, forest, in, northern, New, Mexico, and, its, relationship, to, Holocene, environmental, change, R., Scott, Anderson, a, ,, b, ,, ,, Renata, B., Jass, b, ,1, ,, Jaime, L., Toney, b, ,2, ,, Craig, D., Allen, c, ,, Luz, M., Cisneros-Dozal, d, ,, Marcey, Hess, d, ,, Jeff, Heikoop, d, ,, Julianna, Fessenden, d, a, Center, for, Environmental, Sciences, &, Education, ,, Box, 5694, ,, Northern, Arizona, University, ,, Flagstaff, ,, AZ, 86011, ,, USA, b, Quaternary, Sciences, Program, &, Bilby, Research, ...]"
1,2,"[The, Chihuahueños, Bog, record, extends, to, over, 15,000, cal, yr, BP, .]"
2,3,"[An, Artemisia, steppe, ,, then, an, open, Picea, woodland, grew, around, a, small, pond, until, ca., 11,700, cal, yr, BP, when, Pinus, ponderosa, became, established, .]"
3,4,"[C/N, ratios, ,, δ13C, and, δ15N, values, indicate, both, terrestrial, and, aquatic, organic, matter, was, incorporated, into, the, sediment, .]"
4,5,"[Higher, percentages, of, aquatic, algae, and, elevated, C/N, ratios, indicate, higher, lake, levels, at, the, opening, of, the, Holocene, ,, but, a, wetland, developed, subsequently, as, climate, warmed, .]"
6,6,"[From, ca., 8500, to, 6400, cal, yr, BP, the, pond, desiccated, in, what, must, have, been, the, driest, period, of, the, Holocene, there, .]"
7,7,"[C/N, ratios, declined, to, their, lowest, Holocene, levels, ,, indicating, intense, decomposition, in, the, sediment, .]"
8,8,"[Wetter, conditions, returned, after, 6400, cal, yr, BP, ,, with, conversion, of, the, site, to, a, sedge, bog, as, groundwater, levels, rose, .]"
9,9,"[Higher, charcoal, influx, rates, after, 6400, cal, yr, BP, probably, result, from, greater, biomass, production, rates, .]"
10,10,"[Only, minor, shifts, in, the, overstory, species, occurred, during, the, Holocene, ,, suggesting, that, mixed, conifer, forest, dominated, throughout, the, record, .]"


Article 03  
Skimmed info:  
* `Link`: http://dx.doi.org/10.1130/g35541.1  # No Full access to article
* `Site Name`:  Cahokia region, Illinois, USA
* `Location`:  N38°42'00" - N38°42'00", W90°04'60" - W90°04'60" - Not found on text (yet).
* `Age Span`:   
* `Site Descriptions`: Record from Horseshoe Lake, an oxbow lake in the central Mississippi River valley that is adjacent to the Cahokia site (Illinois, USA), the largest prehistoric settlement north of Mexico.   

In [19]:
# Article 3
order_article('57c5b941cf58f1338eaddb5b')

,sentid,words
67708,1,"[A, record, of, sustained, prehistoric, and, historic, land, use, from, the, Cahokia, region, ,, Illinois, ,, USA, Samuel, E., Munoz1, ,, Sissel, Schroeder2, ,, David, A., Fike3, ,, and, John, W., Williams1, 1Department, of, Geography, ,, University, of, Wisconsin, --, Madison, ,, 550, North, Park, Street, ,, Madison, ,, Wisconsin, 53706, ,, USA, 2Department, of, Anthropology, ,, University, of, Wisconsin, --, Madison, ,, 1180, Observatory, Drive, ,, Madison, ,, Wisconsin, 53706, ,, USA, 3Department, of, Earth, and, Planetary, Sciences, ,, Washington, University, ,, 1, Brookings, Drive, ,, Saint, Louis, ,, Missouri, 63130, ,, USA, ABSTRACT, In, eastern, North, America, ,, ...]"
67709,2,"[Here, we, report, a, high-resolution, and, multiproxy, paleoecological, record, from, Horseshoe, Lake, ,, an, oxbow, lake, in, the, central, Mississippi, River, valley, that, is, adjacent, to, the, Cahokia, site, -LRB-, Illinois, ,, USA, -RRB-, ,, the, largest, prehistoric, settlement, north, of, Mexico, .]"
67710,3,"[Palynological, and, carbon, isotope, data, document, pronounced, vegetation, changes, over, the, past, 1700, yr, driven, primarily, by, land, use, ,, including, 900, yr, -LRB-, 450, --, 1350, CE, -RRB-, of, sustained, prehistoric, human, impacts, .]"
67712,4,"[Rapid, forest, clearance, was, followed, closely, by, the, proliferation, of, indigenous, seed, crops, of, the, Eastern, Agricultural, Complex, beginning, ca., 450, CE, ,, centuries, before, the, emergence, of, Cahokia, at, 1050, CE, .]"
67713,5,"[Agricultural, intensiﬁcation, that, included, the, use, of, maize, -LRB-, Zea, mays, subsp, .]"
67714,6,"[mays, -RRB-, followed, this, initial, clearance, ,, with, peak, land, use, intensity, between, 900, and, 1200, CE, .]"
67715,7,"[A, large, ﬂood, event, ca., 1200, CE, marks, the, onset, of, agricultural, contraction, and, Cahokia, 's, decline, .]"
67716,8,"[Reforestation, follows, the, abandonment, of, the, Cahokia, region, at, ca., 1350, CE, .]"
67718,9,"[The, Horseshoe, Lake, record, thus, indicates, that, regional, agricultural, activity, began, abruptly, at, 450, CE, and, intensiﬁed, over, the, following, centuries, ,, well, before, the, formation, of, Cahokia, and, other, large, prehistoric, settlements, .]"
67719,10,"[The, evidence, that, a, major, ﬂood, coincided, with, the, onset, of, Cahokia, 's, decline, is, noteworthy, ,, but, will, require, corroboration, from, additional, records, .]"


Article 04  
Skimmed info:  
* `Link:`http://www.tandfonline.com/doi/abs/10.1080/00173130902965157
* `Site Name`:  Begbunar
* `Location`:  42°09′ N, 22° 33′ E; 1750 ; in SQL file shows as: `42,°,09,cents,N,,,22,°,33,cents,E`
* `Age Span`:   
* `Site Descriptions`: Located in the central treeless zone of the Osogovo mountains, which are situated at the border between south-western Bulgaria and the north-eastern former Yugoslav Republic of Macedonia.

In [20]:
# Article 4
order_article('58d29193cf58f14928755ba5')

,sentid,words
110659,1,"[Grana, ISSN, :, 0017-3134, -LRB-, Print, -RRB-, 1651-2049, -LRB-, Online, -RRB-, Journal, homepage, :, http://www.tandfonline.com/loi/sgra20, 6, .]"
110660,2,"[Peat-bog, Begbunar, -LRB-, Osogovo, Mountains, ,, southwest, Bulgaria, -RRB-, :, Four, millennia, of, vegetation, history, Maria, Lazarova, ,, Spassimir, Tonkov, ,, Ian, Snowball, &, Elena, Marinova, To, cite, this, article, :, Maria, Lazarova, ,, Spassimir, Tonkov, ,, Ian, Snowball, &, Elena, Marinova, -LRB-, 2009, -RRB-, 6, .]"
110661,3,"[Peat-bog, Begbunar, -LRB-, Osogovo, Mountains, ,, south-west, Bulgaria, -RRB-, :, Four, millennia, of, vegetation, history, ,, Grana, ,, 48:2, ,, 147-149, ,, DOI, :, 10.1080, /, 00173130902965157, To, link, to, this, article, :, http://dx.doi.org/10.1080/00173130902965157, Published, online, :, 09, Jun, 2009, .]"
110662,4,"[Submit, your, article, to, this, journal, Article, views, :, 182, View, related, articles, Citing, articles, :, 5, View, citing, articles, Full, Terms, &, Conditions, of, access, and, use, can, be, found, at, http://www.tandfonline.com/action/journalInformation?journalCode=sgra20, Download, by, :, -LSB-, UW-Madison, -LRB-, GeoDeepDive, -RRB-, -RSB-, Date, :, 22, March, 2017, ,, At, :, 07:39, Grana, ,, 2009, ;, 48, :, 147, --, 149, SCGRA, ONTRIBUTIONS, TO, THE, EUROPEAN, POLLEN, DATABASE, 6, .]"
110665,5,"[Peat-bog, Begbunar, -LRB-, Osogovo, Mountains, ,, south-west, Bulgaria, -RRB-, :, Four, millennia, of, vegetation, history, MARIAEPDB6.Begbunar, -LRB-, Bulgaria, -RRB-, LAZAROVA1, ,, SPASSIMIR, TONKOV2, ,, IAN, SNOWBALL3, &, ELENA, MARINOVA4, 1Institute, of, Botany, ,, Bulgarian, Academy, of, Sciences, ,, Sofia, ,, Bulgaria, ,, 2Laboratory, of, Palynology, ,, Department, of, Botany, ,, Faculty, of, Biology, ,, Sofia, University, ``, St., Kliment, Ohridski, '', ,, Sofia, ,, Bulgaria, ,, 3GeoBiosphere, Science, Centre, ,, Quaternary, Sciences, ,, Department, of, Geology, ,, Lund, University, ,, Lund, ,, Sweden, ,, 4Center, for, Archaeological, Sciences, ,, Katholieke, Universiteit, Leuven, ,, Leuven, ,, Belgium, Site, details, The, peat-bog, ...]"
110666,6,"[This, bog, was, formed, near, a, freshwater, spring, on, a, north-western, slope, ,, which, continues, into, a, steep, ,, deep, ravine, where, isolated, stands, of, beech, -LRB-, Fagus, sylvatica, L., -RRB-, exist, .]"
110667,7,"[The, modern, mountain, vegetation, consists, of, several, vegetation, belts, :, the, oak-hornbeam, belt, -LRB-, up, to, 1000, m, -RRB-, ,, the, compact, beech, belt, -LRB-, 1000, --, 1900, m, -RRB-, with, fragments, of, conifers, -LRB-, Pinus, nigra, J., F., Arnold, and, Abies, alba, Mill, ., -RRB-]"
110668,8,"[and, the, subalpine, belt, ,, which, is, occupied, by, plant, communities, of, Juniperus, sibirica, Burgsd., ,, Vaccinium, myrtillus, L., ,, Bruckenthalia, spiculifolia, -LRB-, Salisb, ., -RRB-]"
110669,9,"[Rchb., ,, Chamaecytisus, absinthioides, Janka, -LRB-, Kuzm, ., -RRB-]"
110663,10,"[,, Nardus, stricta, L., ,, and, scattered, groups, of, Pinus, sylvestris, L., ,, etc., .]"


In [21]:
# Try to see the exact location. Wont show last few words.
pd.DataFrame(order_article('58d29193cf58f14928755ba5').iloc[4])

,110665
sentid,5
words,"[Peat-bog, Begbunar, -LRB-, Osogovo, Mountains, ,, south-west, Bulgaria, -RRB-, :, Four, millennia, of, vegetation, history, MARIAEPDB6.Begbunar, -LRB-, Bulgaria, -RRB-, LAZAROVA1, ,, SPASSIMIR, TONKOV2, ,, IAN, SNOWBALL3, &, ELENA, MARINOVA4, 1Institute, of, Botany, ,, Bulgarian, Academy, of, Sciences, ,, Sofia, ,, Bulgaria, ,, 2Laboratory, of, Palynology, ,, Department, of, Botany, ,, Faculty, of, Biology, ,, Sofia, University, ``, St., Kliment, Ohridski, '', ,, Sofia, ,, Bulgaria, ,, 3GeoBiosphere, Science, Centre, ,, Quaternary, Sciences, ,, Department, of, Geology, ,, Lund, University, ,, Lund, ,, Sweden, ,, 4Center, for, Archaeological, Sciences, ,, Katholieke, Universiteit, Leuven, ,, Leuven, ,, Belgium, Site, details, The, peat-bog, ...]"


Article 05  
Skimmed info:  
* `Link`: http://dx.doi.org/10.1130/0016-7606(1997)109<1306:tfalqp>2.3.co
* `Site Name`:  Cottonwood III, Cottonwood IV, Red Lady, Red Well, Splains
* `Location`:  38°49'50' 106°24'45', 38°49'50' 106°24'45', 38°52'50' 107°2'30', 38°52'50' 107°2'30',38°53'40' 107°3'15', 38°52'28' 107°05', 38°50' 107°4'30', 38°50' 107°4'30', 38°52' 107°2'30', 38°45' 106°50', 38°45' 106°50'
* `Age Span`:   17000 - 400
* `Site Descriptions`:   The lower timberline or lower forest border is the lower elevational limit of a continuous forest belt. In the Rocky Mountains it forms a sharp contrast with shrublands or grasslands in valley bottoms. Trees may extend below the lower timberline in favorable habitats, along streams or on rocky outcrops.  

In [22]:
# Article 5
order_article('57928e07cf58f133d1c26609')

,sentid,words
39606,1,"[Timberline, fluctuations, and, late, Quaternary, paleoclimates, in, the, Southern, Rocky, Mountains, ,, Colorado, Patricia, L., Fall, *, Department, of, Geography, ,, Arizona, State, University, ,, Tempe, ,, Arizona, 85287, ABSTRACT, Pollen, and, plant, macrofossils, from, eight, sedimentary, basins, on, the, west, slope, of, the, Colorado, Rocky, Mountains, document, fluctuations, in, upper, and, lower, timberline, since, the, latest, Pleistocene, .]"
39607,2,"[By, tracking, climatically, sensitive, forest, boundaries, ,, the, moisturecontrolled, lower, timberline, and, the, temperature-controlled, upper, timberline, ,, paleoclimatic, estimates, can, be, derived, from, modern, temperature, and, precipitation, lapse, rates, .]"
39608,3,"[Pollen, data, suggest, that, prior, to, 11, 000, yr, B.P., ,, a, subalpine, forest, dominated, by, Picea, -LRB-, spruce, -RRB-, and, Pinus, -LRB-, pine, -RRB-, grew, 300Ð700, m, below, its, modern, limit, .]"
39609,4,"[The, inferred, climate, was, 2Ð5, ¡, C, cooler, and, had, 7Ð16, cm, greater, precipitation, than, today, .]"
39611,5,"[Abies, -LRB-, fir, -RRB-, increased, in, abundance, in, the, subalpine, forest, around, 11, 000, yr, B.P., ,, probably, in, response, to, cooler, conditions, with, increased, winter, snow, .]"
39612,6,"[Pollen, and, plant, macrofossil, data, demonstrate, that, from, 9000, to, 4000, yr, B.P., the, subalpine, forest, occupied, a, greater, elevational, range, than, it, does, today, .]"
39613,7,"[Upper, timberline, was, 270, m, above, its, modern, limit, ,, suggesting, that, mean, annual, and, mean, July, temperatures, were, 1Ð2, ¡, C, warmer, than, today, .]"
39614,8,"[Intensification, of, the, summer, monsoon, ,, coupled, with, increased, summer, radiation, between, 9000, and, 6000, yr, B.P., ,, raised, mean, annual, precipitation, by, 8Ð11, cm, and, allowed, the, lower, limit, of, the, subalpine, and, montane, forests, to, descend, to, lower, elevations, .]"
39616,9,"[The, lower, forest, border, began, to, retreat, upslope, between, 6000, and, 4000, yr, B.P., in, response, to, drier, conditions, ,, and, the, upper, timberline, descended, after, 4000, yr, B.P., ,, when, temperatures, cooled, to, about, 1, ¡, C, warmer, than, today, .]"
39617,10,"[The, modern, climatic, regime, was, established, about, 2000, yr, B.P., ,, when, the, summer, precipitation, maxima, of, the, early, and, middle, Holocene, were, balanced, by, increased, winter, precipitation, .]"


In [23]:
# Print the exact line where locations are.
pd.DataFrame(order_article('57928e07cf58f133d1c26609').iloc[179])

,39801
sentid,180
words,"[LOCATION, AND, AGE, OF, FOSSIL, STUDY, SITES, Core, Location, Lat, Long, -LRB-, °, N, -RRB-, -LRB-, °, W, -RRB-, Elevation, Depth, -LRB-, m, -RRB-, -LRB-, cm, -RRB-, Age, Laboratory, -LRB-, yr, B.P., -RRB-, number, Cottonwood, III, 38, °, 49, ′, 50, ′, ′, 106, °, 24, ′, 45, ′, ′, 3670, 40, --, 50, 80, --, 90, 120, --, 130, 2460, ±, 130, 5790, ±, 180, 8230, ±, 200, A-4481, A-4482, A-4483, Cottonwood, IV, 38, °, 49, ′, 50, ′, ′, 106, °, 24, ′, 45, ′, ′, 3670, 60, --, 80, 5610, ±, 150, A-4487, 148, --, 168, 8790, ...]"


In [24]:
regex_example = nlp_sentences[nlp_sentences['_gddid'] == '57928e07cf58f133d1c26609']
regex_example = regex_example[regex_example['sentid'] == 180]
regex_example[['words', 'words_as_string', 'dms_regex']]

,words,words_as_string,dms_regex
39801,"[LOCATION, AND, AGE, OF, FOSSIL, STUDY, SITES, Core, Location, Lat, Long, -LRB-, °, N, -RRB-, -LRB-, °, W, -RRB-, Elevation, Depth, -LRB-, m, -RRB-, -LRB-, cm, -RRB-, Age, Laboratory, -LRB-, yr, B.P., -RRB-, number, Cottonwood, III, 38, °, 49, ′, 50, ′, ′, 106, °, 24, ′, 45, ′, ′, 3670, 40, --, 50, 80, --, 90, 120, --, 130, 2460, ±, 130, 5790, ±, 180, 8230, ±, 200, A-4481, A-4482, A-4483, Cottonwood, IV, 38, °, 49, ′, 50, ′, ′, 106, °, 24, ′, 45, ′, ′, 3670, 60, --, 80, 5610, ±, 150, A-4487, 148, --, 168, 8790, ...]","LOCATION,AND,AGE,OF,FOSSIL,STUDY,SITES,Core,Location,Lat,Long,-LRB-,°,N,-RRB-,-LRB-,°,W,-RRB-,Elevation,Depth,-LRB-,m,-RRB-,-LRB-,cm,-RRB-,Age,Laboratory,-LRB-,yr,B.P.,-RRB-,number,Cottonwood,III,38,°,49,′,50,′,′,106,°,24,′,45,′,′,3670,40,--,50,80,--,90,120,--,130,2460,±,130,5790,±,180,8230,±,200,A-4481,A-4482,A-4483,Cottonwood,IV,38,°,49,′,50,′,′,106,°,24,′,45,′,′,3670,60,--,80,5610,±,150,A-4487,148,--,168,8790,±,230,A-4488,Red,Lady,I,38,°,52,′,50,′,′,107,°,2,′,30,′,′,3350,75,--,95,4675,±,155,GX-8350,Red,Lady,II,38,°,52,′,50,′,′,107,°,2,′,30,′,′,3350,10,--,20,45,--,55,74,--,84,100,±,70,2520,±,90,4680,±,120,A-4484,A-4485,A-4486,Red,Well,38,°,53,′,40,′,′,107,°,3,′,15,′,′,3290,80,--,100,2805,±,160,GX-8351,Copley,II,38,°,52,′,28,′,′,107,°,05,′,3250,25,--,35,62,--,72,102,--,112,125,--,135,155,--,165,181,--,191,212,--,222,1570,±,150,5690,±,150,6900,±,220,8210,±,355,10325,±,465,12810,±,750,15370,±,880,A-4479,A-4480,GX-7906,GX-12995,GX-12996,GX-12997,GX-7907,Splains,II,38,°,50,′,107,°,4,′,30,′,′,3160,45,--,55,90,--,100,190,--,200,266,--,286,291,--,292,3880,±,170,5040,±,140,8750,±,220,11040,±,240,12020,±,530,A-4408,A-4407,A-4409,A-2535,AA-642,Splains,Gulch,I,38,°,50,′,107,°,4,′,30,′,′,3150,34,--,48,80,--,98,134,--,148,190,--,200,4440,±,120,5620,±,140,7340,±,240,8560,±,600,A-4410,A-2534,A-4411,A-2424,Ironbog,38,°,52,′,107,°,2,′,30,′,′,2920,48,--,50,58,--,66,134,--,136,170,195,--,200,218,--,220,2640,±,90,3105,±,155,6300,±,80,7700,±,70,7100,±,255,8260,±,80,AA-1125,GX-7905,AA-1126,AA-1127,GX-7339,AA-1128,Alkali,Basin,I,38,°,45,′,106,°,50,′,2750,137,3480,±,135,GX-6796,300,7857,±,260,GX-6795,410,12975,±,430,GX-6794,Alkali,Basin,II,38,°,45,′,106,°,50,′,2750,533,--,549,17440,±,1200,A-2421,conifers,,,Picea,and,Abies,,,drop,to,levels,found,above,treeline,in,the,krummholz,zone,today,,,and,Pinus,pollen,percentages,and,accumulation,increase,-LRB-,see,Figs.,6A,and,7A,-RRB-,.","[38,°,49,′,50,′,′,106,°,24,′,45,′,′,, 38,°,49,′,50,′,′,106,°,24,′,45,′,′,, 38,°,52,′,50,′,′,107,°,2,′,30,′,′,, 38,°,52,′,50,′,′,107,°,2,′,30,′,′,, 38,°,53,′,40,′,′,107,°,3,′,15,′,′,, 38,°,52,′,28,′,′,107,°,05,′,, 38,°,50,′,107,°,4,′,30,′,′,, 38,°,50,′,107,°,4,′,30,′,′,, 38,°,52,′,107,°,2,′,30,′,′,, 38,°,45,′,106,°,50,′,, 38,°,45,′,106,°,50,′,]"


In [25]:
# Find the article in the bibliography by gddid to retrieve Article Name

bibliography[bibliography['_gddid'] == '57928e07cf58f133d1c26609']

,_type,_id,publisher,title,journal.name.name,author,year,number,volumne,_gddid,type,pages,link_url,link_type
93,doi,10.1130/0016-7606(1997)109<1306:tfalqp>2.3.co;2,GSA,"Timberline fluctuations and late Quaternary paleoclimates in the Southern Rocky Mountains, Colorado",Geological Society of America Bulletin,"Fall, Patricia L.",1997,10,NaN,57928e07cf58f133d1c26609,article,1306--1320,http://dx.doi.org/10.1130/0016-7606(1997)109<1306:tfalqp>2.3.co;2,publisher


## Linking Neotoma DB to extract locations

In [26]:
neotoma = pd.read_csv("../Do_not_commit_data/data-1590729612420.csv")
neotoma = neotoma[['siteid', 'sitename', 'longitudeeast', 'latitudenorth', 'longitudewest', 'latitudesouth', 'sitedescription', 'doi']]

neotoma.head(3)

,siteid,sitename,longitudeeast,latitudenorth,longitudewest,latitudesouth,sitedescription,doi
0,10330,Lac du Sommet,-70.66468,47.71662,-70.66573,47.71382,"The small shallow Lac du Sommet (0.02 km2, 4 m maximum depth, elevation of 830 m a.s.l., 47°43′N, 70°40′W) is located in the boreal forest north of the St Lawrence Estuary (Figure 1). It is of glacial origin and situated on granitic-gneissic bedrock of the Canadian Precambrian Shield in the Laurentian Mountains.",10.1177/0959683611400199
1,10330,Lac du Sommet,-70.66468,47.71662,-70.66573,47.71382,"The small shallow Lac du Sommet (0.02 km2, 4 m maximum depth, elevation of 830 m a.s.l., 47°43′N, 70°40′W) is located in the boreal forest north of the St Lawrence Estuary (Figure 1). It is of glacial origin and situated on granitic-gneissic bedrock of the Canadian Precambrian Shield in the Laurentian Mountains.",10.1177/0959683611400199
2,1729,Myrtle Lake,-93.37853,47.98645,-93.39207,47.97876,"Lake surrounded by peatland. Physiography: Red Lake lowlands. Surrounding vegetation: Sphagnum, Piceto-Chamaedophnetum.",10.1139/b68-190


In [27]:
nlp_bib = nlp_sentences.merge(bibliography, on = '_gddid')
nlp_bib = nlp_bib.rename(columns={'_id': 'doi'})
nlp_bib.head(3)

,_gddid,sentid,wordidx,words,part_of_speech,special_class,lemmas,word_type,word_modified,words_as_string,dms_regex,_type,doi,publisher,title,journal.name.name,author,year,number,volumne,type,pages,link_url,link_type
0,54b43266e138239d8684efed,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, ...]","[Available, online, at, www.sciencedirect.com, Quaternary, Research, 69, -LRB-, 2008, -RRB-, 263, --, 275, www.elsevier.com/locate/yqres, Development, of, the, mixed, conifer, forest, in, northern, New, Mexico, and, its, relationship, to, Holocene, environmental, change, R., Scott, Anderson, a, ,, b, ,, ,, Renata, B., Jass, b, ,1, ,, Jaime, L., Toney, b, ,2, ,, Craig, D., Allen, c, ,, Luz, M., Cisneros-Dozal, d, ,, Marcey, Hess, d, ,, Jeff, Heikoop, d, ,, Julianna, Fessenden, d, a, Center, for, Environmental, Sciences, &, Education, ,, Box, 5694, ,, Northern, Arizona, University, ,, Flagstaff, ,, AZ, 86011, ,, USA, b, Quaternary, Sciences, Program, &, Bilby, Research, ...]","[JJ, NN, IN, NNP, NNP, NNP, CD, -LRB-, CD, -RRB-, CD, :, CD, NNS, NN, IN, DT, JJ, NN, NN, IN, JJ, NNP, NNP, CC, PRP$, NN, TO, NNP, JJ, NN, NNP, NNP, NNP, DT, ,, NN, ,, ,, NNP, NNP, NNP, NN, CD, ,, NNP, NNP, NNP, NN, CD, ,, NNP, NNP, NNP, NN, ,, NNP, NNP, NNP, NN, ,, NNP, NNP, NN, ,, NNP, NNP, NN, ,, NNP, NNP, NN, DT, NNP, IN, NNP, NNPS, CC, NNP, ,, NNP, CD, ,, NNP, NNP, NNP, ,, NNP, ,, NNP, CD, ,, NNP, NN, NNP, NNP, NNP, CC, NNP, NNP, ...]","[O, O, O, O, O, O, NUMBER, O, DATE, O, NUMBER, O, NUMBER, O, O, O, O, O, O, O, O, O, LOCATION, LOCATION, O, O, O, O, O, O, O, PERSON, PERSON, PERSON, O, O, O, O, O, PERSON, PERSON, PERSON, O, NUMBER, O, PERSON, PERSON, PERSON, O, NUMBER, O, PERSON, PERSON, PERSON, O, O, PERSON, PERSON, PERSON, O, O, PERSON, PERSON, O, O, PERSON, PERSON, O, O, PERSON, PERSON, O, O, ORGANIZATION, ORGANIZATION, ORGANIZATION, ORGANIZATION, ORGANIZATION, ORGANIZATION, O, O, NUMBER, O, ORGANIZATION, ORGANIZATION, ORGANIZATION, O, LOCATION, O, LOCATION, NUMBER, O, ORGANIZATION, ORGANIZATION, ORGANIZATION, ORGANIZATION, ORGANIZATION, ORGANIZATION, ORGANIZATION, ORGANIZATION, ...]","[available, online, at, www.sciencedirect.com, Quaternary, Research, 69, -lrb-, 2008, -rrb-, 263, --, 275, www.elsevier.com/locate/yqres, development, of, the, mixed, conifer, forest, in, northern, New, Mexico, and, its, relationship, to, Holocene, environmental, change, R., Scott, Anderson, a, ,, b, ,, ,, Renata, B., Jass, b, ,1, ,, Jaime, L., Toney, b, ,2, ,, Craig, D., Allen, c, ,, Luz, M., Cisneros-Dozal, d, ,, Marcey, Hess, d, ,, Jeff, Heikoop, d, ,, Julianna, Fessenden, d, a, Center, for, Environmental, Sciences, &, Education, ,, Box, 5694, ,, Northern, Arizona, University, ,, Flagstaff, ,, AZ, 86011, ,, USA, b, Quaternary, Sciences, Program, &, Bilby, Research, ...]","[dep, dep, dep, dep, dep, dep, dep, , dep, , dep, , dep, dep, dep, dep, dep, dep, dep, dep, dep, dep, dep, dep, cc, dep, dep, dep, dep, dep, dep, dep, dep, dep, dep, , dep, , , dep, dep, dep, dep, dep, , dep, dep, dep, dep, dep, , dep, dep, dep, dep, , dep, dep, dep, dep, , dep, dep, dep, , dep, dep, dep, , dep, dep, dep, dep, dep, dep, dep, dep, cc, dep, , dep, dep, , dep, dep, dep, , dep, , dep, dep, , dep, dep, dep, dep, dep, cc, dep, dep, ...]","[218, 218, 218, 218, 218, 218, 218, 0, 218, 0, 218, 0, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 0, 218, 0, 0, 218, 218, 218, 218, 218, 0, 218, 218, 218, 218, 218, 0, 218, 218, 218, 218, 0, 218, 218, 218, 218, 0, 218, 218, 218, 0, 218, 218, 218, 0, 218, 218, 218, 218, 218, 218, 218, 218, 218, 218, 0, 218, 218, 0, 218, 218, 218, 0, 218, 0, 218, 2

In [28]:
print(nlp_bib.shape)

(111965, 24)


In [29]:
nlp_bib[['words_as_string', 'journal.name.name', 'dms_regex']].head(3)

,words_as_string,journal.name.name,dms_regex
0,"Available,online,at,www.sciencedirect.com,Quaternary,Research,69,-LRB-,2008,-RRB-,263,--,275,www.elsevier.com/locate/yqres,Development,of,the,mixed,conifer,forest,in,northern,New,Mexico,and,its,relationship,to,Holocene,environmental,change,R.,Scott,Anderson,a,,,b,,,,,Renata,B.,Jass,b,,1,,,Jaime,L.,Toney,b,,2,,,Craig,D.,Allen,c,,,Luz,M.,Cisneros-Dozal,d,,,Marcey,Hess,d,,,Jeff,Heikoop,d,,,Julianna,Fessenden,d,a,Center,for,Environmental,Sciences,&,Education,,,Box,5694,,,Northern,Arizona,University,,,Flagstaff,,,AZ,86011,,,USA,b,Quaternary,Sciences,Program,&,Bilby,Research,Center,,,Box,6013,,,Northern,Arizona,University,,,Flagstaff,,,AZ,86011,,,USA,c,U.S.,Geological,Survey,,,Jemez,Mountains,Field,Station,,,Bandelier,National,Monument,,,HCR-1,,,Box,1,,,#,15,,,Los,Alamos,,,NM,87544,,,USA,d,Earth,&,Environmental,Sciences,Division,,,Hydrology,,,Geochemistry,&,Geology,Group,,,EES-6,,,MS-D462,,,Los,Alamos,National,Laboratory,,,Los,Alamos,,,NM,87545,,,USA,Received,28,April,2007,Available,online,29,January,2008,Abstract,Chihuahueños,Bog,-LRB-,2925,m,-RRB-,in,the,Jemez,Mountains,of,northern,New,Mexico,contains,one,of,the,few,records,of,late-glacial,and,postglacial,development,of,the,mixed,conifer,forest,in,southwestern,North,America,.",Quaternary Research,[]
1,"The,Chihuahueños,Bog,record,extends,to,over,15,000,cal,yr,BP,.",Quaternary Research,[]
2,"An,Artemisia,steppe,,,then,an,open,Picea,woodland,grew,around,a,small,pond,until,ca.,11,700,cal,yr,BP,when,Pinus,ponderosa,became,established,.",Quaternary Research,[]


Let's now try to extract Sitenames using Neotoma database. For this, I will group by doi and then, list all sitenames that are mentioned in each doi.

In [30]:
by_doi = neotoma.groupby('doi')
neotoma_sitenames = by_doi.apply(lambda x: list(x['sitename']))
neotoma_siteid = by_doi.apply(lambda x: list(x['siteid']))
neotoma_longeast = by_doi.apply(lambda x: list(x['longitudeeast']))
neotoma_latnorth = by_doi.apply(lambda x: list(x['latitudenorth']))

neotoma_joined_df = neotoma_sitenames.to_frame('sitenames').join(neotoma_siteid.to_frame('siteid')).join(neotoma_longeast.to_frame('longeast')).join(neotoma_latnorth.to_frame('latnorth'))
neotoma_joined_df = neotoma_joined_df.reset_index()

#neotoma_joined_df['intlongeast'] = [int(i) for neotoma_joined_df['longeast'][i]]

#print(neotoma_joined_df['longeast'][1].astype(int))
#[round(x) for x in myList]
neotoma_joined_df.head(5)


,doi,sitenames,siteid,longeast,latnorth
0,0.1139/e80-122,"[East Baltic Bog, Mermaid Bog, Portage Bog]","[763, 1684, 1903]","[-62.15, -63.025, -64.075]","[46.408333, 46.25, 46.673611]"
1,10.1002/(SICI)1099-1417(199607/08)11:4<327::AID-JQS257>3.0.CO;2-G,[Lundin Tower],[3256],[-2.97206],[56.21529]
2,10.1002/(SICI)1099-1417(199701/02)12:1<15::AID-JQS283>3.0.CO;2-T,"[Spruce Pond, Spruce Pond, Spruce Pond]","[2508, 2508, 2508]","[-74.18236, -74.18236, -74.18236]","[41.237770000000005, 41.237770000000005, 41.237770000000005]"
3,10.1002/(SICI)1099-1417(199801/02)13:1<43::AID-JQS348>3.0.CO;2-G,"[Pantano de Genagra, Pantano de Genagra]","[14030, 14030]","[-76.60428, -76.60428]","[2.4751, 2.4751]"
4,10.1002/1099-1417(200010)15:7<687::AID-JQS551>3.0.CO;2-6,"[Laguna Loma Linda, Laguna Loma Linda]","[13971, 13971]","[-73.35661, -73.35661]","[3.30805, 3.30805]"


From Neotoma DB, we have obtained a dataframe grouped by DOI that lists each `sitename`, `siteid`, `longitude east`, and `latitunde north`. Now, let's join this information to the NLP_Bib dataframe. For now, let's do to the one that coordinates have been obtained with REGEX just to see if some of the numbers and sitenames make sense. 

In [31]:
nlp_bib_neotoma = nlp_bib.merge(neotoma_joined_df, on = 'doi')
print(nlp_bib_neotoma.shape)

(106640, 28)


In [32]:
# Filter df to just frames that has REGEX longer than one
mask = (nlp_bib_neotoma['dms_regex'].str.len() > 0)
nlp_bib_neotoma_regex = nlp_bib_neotoma.loc[mask]
nlp_bib_neotoma_regex[['words','dms_regex']].head(100)

print(nlp_bib_neotoma_regex.shape)

(93, 28)


In [33]:
nlp_bib_neotoma_regex = nlp_bib_neotoma_regex[['_gddid', 'sentid', 'doi', 'words', 'words_as_string', 'dms_regex', 'sitenames', 'latnorth', 'longeast']]
nlp_bib_neotoma_regex = nlp_bib_neotoma_regex.reset_index()
nlp_bib_neotoma_regex.head(1)

,index,_gddid,sentid,doi,words,words_as_string,dms_regex,sitenames,latnorth,longeast
0,3314,578a1f6ecf58f1b54b810cd9,94,10.1139/b85-070,"[Lake, Hope, Simpson, -LRB-, 52, ', 27, ', N, ,, 56, ', 26, ', W, ,, elevation, 295, m, -RRB-, lies, 20, km, southwest, of, Port, Hope, Simpson, in, a, steep-sided, eastwest-trending, valley, that, is, drained, by, the, St., Lewis, River, .]","Lake,Hope,Simpson,-LRB-,52,',27,',N,,,56,',26,',W,,,elevation,295,m,-RRB-,lies,20,km,southwest,of,Port,Hope,Simpson,in,a,steep-sided,eastwest-trending,valley,that,is,drained,by,the,St.,Lewis,River,.","[52,',27,',N,,,56,',26,',W]","[Lake Hope Simpson, 18 [Moraine Lake]]","[52.45, 52.266667]","[-56.433333, -58.05]"


In [34]:
any( k in s for k in keywords )

NameError: name 'keywords' is not defined

We can see that most of the REGEX indeed pulled out the same locations for latitude and longitudes as in the Neotoma database.

In [35]:
nlp_bib_neotoma[['_gddid', 'sentid', 'doi', 'words', 'dms_regex', 'sitenames', 'latnorth', 'longeast']].head(2)

,_gddid,sentid,doi,words,dms_regex,sitenames,latnorth,longeast
0,54b43266e138239d8684efed,1,10.1016/j.yqres.2007.12.002,"[Available, online, at, www.sciencedirect.com, Quaternary, Research, 69, -LRB-, 2008, -RRB-, 263, --, 275, www.elsevier.com/locate/yqres, Development, of, the, mixed, conifer, forest, in, northern, New, Mexico, and, its, relationship, to, Holocene, environmental, change, R., Scott, Anderson, a, ,, b, ,, ,, Renata, B., Jass, b, ,1, ,, Jaime, L., Toney, b, ,2, ,, Craig, D., Allen, c, ,, Luz, M., Cisneros-Dozal, d, ,, Marcey, Hess, d, ,, Jeff, Heikoop, d, ,, Julianna, Fessenden, d, a, Center, for, Environmental, Sciences, &, Education, ,, Box, 5694, ,, Northern, Arizona, University, ,, Flagstaff, ,, AZ, 86011, ,, USA, b, Quaternary, Sciences, Program, &, Bilby, Research, ...]",[],"[Chihuahueños Bog, Chihuahueños Bog]","[36.0491, 36.0491]","[-106.50963, -106.50963]"
1,54b43266e138239d8684efed,2,10.1016/j.yqres.2007.12.002,"[The, Chihuahueños, Bog, record, extends, to, over, 15,000, cal, yr, BP, .]",[],"[Chihuahueños Bog, Chihuahueños Bog]","[36.0491, 36.0491]","[-106.50963, -106.50963]"


In [36]:
#nlp_bib_neotoma['words_l'] = nlp_bib_neotoma['words'].apply(lambda col: col.str.lower())

nlp_bib_neotoma['words_l']  = [s.lower() for s in nlp_bib_neotoma['words']]
nlp_bib_neotoma[['words', 'words_l']].head(2)



AttributeError: 'list' object has no attribute 'lower'

In [58]:
a = nlp_bib_neotoma_regex['sitenames'][48]
a = [x.upper() for x in a]

In [59]:
b = nlp_bib_neotoma_regex['words'][48]

In [60]:
list(set(a).intersection(b))

['UDDELERMEER', 'BLEEKEMEER']

In [56]:
print(type(nlp_bib_neotoma['sitenames'][48]))
print(type(nlp_bib_neotoma['words'][48]))
print(type(nlp_bib_neotoma['words_as_string'][48]))

<class 'list'>
<class 'list'>
<class 'str'>


In [43]:
#for i in range(len(nlp_bib_neotoma.index)):
#    nlp_bib_neotoma['sitename_in_wordsstr'] = any(item in nlp_bib_neotoma['words_as_string'][i] for item in nlp_bib_neotoma['sitenames'][i])
    
    
    

In [265]:
#nlp_bib_neotoma.head(299)